<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStardust/blob/main/Securely_add_and_use_your_Google_OAuth_credentials_in_your_Colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-decouple

In [ ]:
from google.colab import files
files.upload()  # Upload your .env file containing the client_id and client_secret

In [ ]:
from decouple import config

client_id = config('GOOGLE_CLIENT_ID')
client_secret = config('GOOGLE_CLIENT_SECRET')

In [ ]:
from flask import Flask, redirect, url_for, session
from authlib.integrations.flask_client import OAuth
import os
from decouple import config

app = Flask(__name__)
app.secret_key = os.urandom(24)  # Replace with your own secret key

oauth = OAuth(app)
oauth.register(
    name='google',
    client_id=config('GOOGLE_CLIENT_ID'),  # Load from .env
    client_secret=config('GOOGLE_CLIENT_SECRET'),  # Load from .env
    authorize_url='https://accounts.google.com/o/oauth2/auth',
    authorize_params=None,
    access_token_url='https://accounts.google.com/o/oauth2/token',
    access_token_params=None,
    refresh_token_url=None,
    client_kwargs={'scope': 'openid profile email'},
    server_metadata_url='https://accounts.google.com/.well-known/openid-configuration'
)

@app.route('/')
def index():
    return 'Welcome to the OAuth2 example!'

@app.route('/login')
def login():
    redirect_uri = url_for('authorize', _external=True)
    return oauth.google.authorize_redirect(redirect_uri)

@app.route('/logout')
def logout():
    for key in list(session.keys()):
        session.pop(key)
    return 'You have been logged out.'

@app.route('/login/authorize')
def authorize():
    token = oauth.google.authorize_access_token()
    user_info = oauth.google.parse_id_token(token)

    return 'Logged in as: ' + user_info['email']

if __name__ == '__main__':
    app.run(debug=True)